# Dependency Injection

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.13.0"
#r "nuget: Microsoft.DotNet.Interactive, 1.0.0-beta.24229.4"
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Debug, 8.0.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.13.0-alpha"
#r "nuget: Microsoft.Extensions.Http, 8.0.0"
#r "nuget: Microsoft.Extensions.Http.Resilience"
#r "nuget: Microsoft.Extensions.Compliance.Redaction"



In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel.Plugins.OpenApi;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using Microsoft.Extensions.Compliance.Redaction;

# Seperate Builder

In [ ]:
var builder = Kernel.CreateBuilder();


## Add Logging

In [ ]:
builder.Services.AddLogging(b => b.AddConsole().SetMinimumLevel(LogLevel.Trace));

In [ ]:
Kernel kernel = builder
    .AddOpenAIChatCompletion("gpt-4o", Environment.GetEnvironmentVariable("OAI_KEY")!)
    .Build();


In [ ]:
var settings = new OpenAIPromptExecutionSettings() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
var chatService = kernel.GetRequiredService<IChatCompletionService>();
ChatHistory chat = new();

## Add Settings and Kernel to ChatService
Show in Rider.

In [ ]:
while (true)
{
    Console.WriteLine("Q: ");
    var message = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Your prompt");
    chat.AddUserMessage(message);
    Console.WriteLine($"User: {message}");
    
    var response = await chatService.GetChatMessageContentAsync(chat, settings, kernel); // <-- Added settings and kernel

    Console.WriteLine(response);
    chat.Add(response);
}

## Import from everywhere
- `kernel.ImportPluginFromFunctions();`
- `kernel.ImportPluginFromObject();`
- `kernel.ImportPluginFromType<>();`
- `kernel.ImportPluginFromOpenApiAsync()`
## Add Websearch to our AI

In [ ]:
#pragma warning disable SKEXP0050
kernel.ImportPluginFromObject(new Microsoft.SemanticKernel.Plugins.Web.WebSearchEnginePlugin(
    new BingConnector(Environment.GetEnvironmentVariable("BING_API_KEY")!)));

In [ ]:
while (true)
{
    Console.WriteLine("Q: ");
    var message = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Your prompt");
    chat.AddUserMessage(message);
    Console.WriteLine($"User: {message}");
    
    var response = await chatService.GetChatMessageContentAsync(chat, settings, kernel);

    Console.WriteLine(response);
    chat.Add(response);
}

## Any other service with DI

In [ ]:
builder.Services.ConfigureHttpClientDefaults( b => {
    b.AddStandardResilienceHandler();
    b.RedactLoggedHeaders(["Authorization"]); // --> Hide API key
});
builder.Services.AddRedaction();